In [1]:
import pandas as pd
import re 
import numpy as np
import matplotlib.pyplot as plt
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.tree import plot_tree

#### Источники данных

In [2]:
token_vk = "vk1.a.R9Pc7uDhuLDOZ7H5Vexi2eR5Tqtls9csgt2XmjPdqunaOyph1ZRHWoQpE9dpgASVmO6Ih_UIVS2tZZqpl8NgiGbY6QgTVt2c7YwALObN_diJ9I69ts5WDiMdVSaxeOpgj9MiGbeAbub2Y1j95irpwJ-_hJX2U3GWoWMHmGEufTIx72-IVbX61M2RFXwnSNVR"
id_sankt_pet = '-187956888'
id_auto = '-93614683'
id_moscow = '-137324015'
mas_id = [id_sankt_pet,id_auto,id_moscow]

#### Получение данных

In [3]:
from vk_api import VkApi
import vk_api

token = token_vk

def main(offset: int, token: str, group_id: str):
    vk = vk_api.VkApi(token=token) # авторизация через токен (желательно)
    api = vk.get_api()
    posts = api.wall.get(owner_id = group_id, offset = offset, count=100)['items']
    posts_strings = [post['text'] for post in posts]
    num_like = []
    comments_strings = []
    for post in posts:
        comments = api.wall.getComments(owner_id = group_id, post_id=post['id'], count=100)['items']
        comments_strings.append([comment['text'] for comment in comments])
        itemID = post['id']
        isLiked = api.likes.getList(
        type = 'post', 
        owner_id = group_id, 
        item_id = itemID               
        )
        num_like.append(isLiked['count'])
    return posts_strings, comments_strings, num_like


combo_list_posts = []
combolist_comments = []
combolist_like_count = []

for group_id in mas_id:
    for i in range(0, 301, 100):
        try:
            rzd_posts, comments_strings_rzd, rzd_like_count = main(offset = i, token = token, group_id = group_id)
            combo_list_posts.extend(rzd_posts)
            combolist_comments.extend(comments_strings_rzd)
            combolist_like_count.extend(rzd_like_count)
            print(group_id,i)
        except:
            print('Постов больше нет на смещении: ', i)


-187956888 0
-187956888 100
-187956888 200
-187956888 300
-93614683 0
-93614683 100
-93614683 200
-93614683 300
-137324015 0
-137324015 100
-137324015 200
-137324015 300


In [15]:
#категории постов
mas_category = ['транспорт санкт-перебурга' for i in range(400)]
mas_category.extend(['автомобильный' for i in range(400)])
mas_category.extend(['транспорт москвы' for i in range(400)])

In [22]:
#количество комментариев в каждом посту
combolist_comments_count = [len(comments) for comments in combolist_comments]

In [23]:
#датасет с данными
data = pd.DataFrame({'посты':combo_list_posts,'комментарии':combolist_comments_count,'лайки':combolist_like_count})

In [24]:
data

,посты,комментарии,лайки
0,Уважаемые петербуржцы! \n \nВ официальной груп...,0,154
1,"#усиление_маршрутов\n\n9 апреля, в Вербное вос...",7,30
2,Для повышения уровня безопасности более чем на...,12,14
3,Трамвайный парк №3 Горэлектротранса отмечает 1...,6,45
4,Почти два года Комитет по транспорту исполняет...,4,5
...,...,...,...
1195,#Наземныйтранспорт\nС СЕГОДНЯШНЕГО ДНЯ ДВА ЭКС...,21,221
1196,#Наземныйтранспорт\nО ТРАМВАЙНОЙ ЛИНИИ НА ПРОС...,17,125
1197,#Метро\nСОБЯНИН: СО СЛЕДУЮЩЕГО ГОДА В СЕРИЮ ПО...,31,159
1198,#Речнойтранспорт\nЮЖНЫЙ РЕЧНОЙ ВОКЗАЛ. ДО ОТКР...,3,137


#### EDA и инженерия данных

In [25]:
data=data.drop_duplicates()#удаление дубликатов

In [26]:
data.isnull().sum()#переменные с пропуском

посты          0
комментарии    0
лайки          0
dtype: int64

#### Токенизация

In [3]:
data = data.astype({'посты': np.str})

C:\Users\Nikita\AppData\Local\Temp\ipykernel_9036\1167223233.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = data.astype({'посты': np.str})


In [4]:
stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()
lemmatizer = WordNetLemmatizer()
tknzr = TweetTokenizer()
tokenize_document = []

for document in data['посты']:
    document = tknzr.tokenize(document)
    document = [word.lower() for word in document]
    document = [word for word in document if word not in stop_words]
    document = [lemmatizer.lemmatize(word) for word in document]
    document = [word for word in document if word not in ':;.,!?\'\"/)( ']
    document = [morph.normal_forms(word)[0] for word in document]
    tokenize_document.append(document)


In [5]:
token_text = [' '.join(i) for i in tokenize_document]
token_text

['уважаемый петербуржец официальный группа комитет транспорт участиться случай комментарий отвечать сторонний пользователь имя группа который мочь ассоциироваться комитет транспорт злоумышленник давать ложный информация деятельность комитет маршрут общественный транспорт прочий вводить заблуждение напоминать официальный комментарий имя комитет транспорт спб гку « организатор перевозка » правительство санкт-петербург писаться шапка « комитет транспорт » галочка верификация остальной пост человек сообщество маскироваться комитет транспорт ассоциироваться представитель комитет транспорт являться частный мнение сторонний человек иметь отношение комитет транспорт ❗ ️просить проявлять бдительность реагировать такой пост вступать переписка ‼ ️напоминать правило группа просить использовать оскорбление относиться уважительно администрация участник группа зависимость « степень тяжесть провинность » нарушитель выдать билет « чёрный список »',
 '#усиление_маршрут 9 апрель вербный воскресение удобс

In [6]:
#добавление в датафрейм обработанного текста 
data['token_text'] = token_text

In [80]:
#tf = количество нашего слова в документе
#idf = ln[(1+количество документов)/(1+количество док-ов где встречается наше слово)]+1
#tf*idf
#потом происходит нормализация по евклидовой норме, idf/sqrt(Σtf**2) 

In [18]:
vectorizer = TfidfVectorizer(min_df = 10)
vectors = vectorizer.fit_transform(token_text)
fichi = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
fichi

,00,000,01,10,100,1000,11,110,115,116,...,этап,это,этот,южный,юрлицо,являться,январь,яндекс,японский,ёмкость
0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.078195,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.00000,0.089844,0.054485,0.0,0.053924,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.134969,0.0,0.000000,0.000000,0.075365,0.0,0.0
2,0.0,0.0,0.00000,0.054926,0.066620,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.082056,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.085652,0.086981,...,0.0,0.035376,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.035961,0.000000,0.000000,0.0,0.059017,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
1194,0.0,0.0,0.07733,0.000000,0.000000,0.0,0.054104,0.0,0.000000,0.000000,...,0.0,0.000000,0.051575,0.000000,0.0,0.000000,0.055657,0.000000,0.0,0.0
1195,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
1196,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.378503,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [35]:
fichi['лайки'] = data['лайки']
y = data['комментарии']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
fichi, y, test_size=0.3, random_state=42)

##### Выбор инструмента и подбор параметров

In [37]:
parametrs={'max_depth': [2,5,7,10],'n_estimators' : [10,20,30]}

regressor = RandomForestRegressor()

#### Обучение модели и проверка метрик

In [38]:
grid_search_cv_reg=GridSearchCV(regressor, parametrs, cv=5)#добавляем еще кросс-валидацию с пятью фолдами
grid_search_cv_reg.fit(X_train, y_train)
best_reg = grid_search_cv_reg.best_estimator_

In [40]:
grid_search_cv_reg.best_params_#лучшие параметры для нашей модели

{'max_depth': 10, 'n_estimators': 30}

In [45]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print(f"MSE = {mean_squared_error(y_test,best_reg.predict(X_test))}")
print(f"MAE = {mean_absolute_error(y_test,best_reg.predict(X_test))}")

MSE = 134.41213043078176
MAE = 7.8073822184791295
